<h1 align="center">Natural Language Processing (NLP) Project, Part 4</h1>
<h2 align="center">Machine Learning Classifiers - Random Forest model</h2>
<h3 align="center">Rositsa Chankova</h3>

### Random Forest model is an ensemble learning method that constructs a collection of decision trees and then aggregates the predictions of each tree to determine the final prediction. List of benefits:

1. **Can be used for classification or regression**
2. **Easily handles outliers, missing values, etc.**
3. **Accepts various types of inputs (continuous, ordinal, etc.)**
4. **Less likely to overfit**
5. **Output feature importance**

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TfidfVectorizer is used which show how frequently a word occurs in that text message 
# relativly to other text messages
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head() #does not include labels

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### This all the data + newly created features

### Explore RandomForestClassifier Attributes & Hyperparameters

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'n_features_', 'predict', 'predict_log_proba', 'predict_proba', 'score',

### Important Hyprparameters:
1. **max_depth - how deep the RF is**
2. **n_estimators - how much DT will be build in the RF**

### Explore RandomForestClassifier through Cross-Validation

In [4]:
from sklearn.model_selection import KFold, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
rf = RandomForestClassifier(n_jobs=-1) # -1 - the processes to be run in parallel 

k_fold = KFold(n_splits=5) # how many splits

cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97666068, 0.97935368, 0.97574124, 0.96585804, 0.97574124])

### Explore RandomForestClassifier through Holdout Set

In [6]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

### RF can give feature importance - here the 'body_len' is the most important feature

In [9]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.062156311460262244, 'body_len'),
 (0.04071493134316473, 7350),
 (0.03959116748499896, 1803),
 (0.03776553308112209, 4796),
 (0.02711464541864713, 5724),
 (0.02184245915087901, 3134),
 (0.020181866894236865, 6285),
 (0.019703907013620427, 2031),
 (0.018909803283795826, 5988),
 (0.015936265829834602, 6746)]

### Prediction

In [10]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [11]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.52 / Accuracy: 0.936


![alt text](recall_precision.png)

### Build Grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

In [12]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [13]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 0.969 / Recall: 0.209 / Accuracy: 0.894
Est: 10 / Depth: 20 ---- Precision: 0.952 / Recall: 0.541 / Accuracy: 0.935
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.649 / Accuracy: 0.953
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.736 / Accuracy: 0.965
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.209 / Accuracy: 0.895
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.514 / Accuracy: 0.935
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.635 / Accuracy: 0.952
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.736 / Accuracy: 0.965
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.189 / Accuracy: 0.892
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.527 / Accuracy: 0.937
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.662 / Accuracy: 0.955
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.73 / Accuracy: 0.964


### As the Depth increases the Recall increases, so model become more agressive

### Comparing the performance of  TF-IDF Vectorizer and  CountVectorizer

In [14]:
# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [15]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [16]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,3.072620,0.238697,0.230622,0.040476,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.979354,0.978456,0.977538,0.965858,0.971249,0.974491,0.005171,1
10,22.951517,0.281132,0.392118,0.132733,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978456,0.978456,0.974843,0.969452,0.969452,0.974132,0.004042,2
7,22.375834,0.045390,0.385816,0.041838,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.976661,0.973944,0.965858,0.973944,0.973593,0.004128,3
8,39.567154,1.877924,0.412897,0.086595,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.977558,0.975741,0.966757,0.970350,0.973593,0.004320,4
11,31.493144,4.336165,0.265748,0.016177,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.975763,0.973944,0.967655,0.971249,0.973413,0.003717,5


In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

### The best performing models are the ones with deepest DT

### TF-IDF Vectorizer is doing a litle bit better that CountVectoriser